图片转向量的缺点：  
- 空间关系丢失
- 模型大小很大

In [1]:
import mxnet as mx
from mxnet import nd
#对于w0，数据格式是batch*channel*height*widht,其中batch和channel都是1
w0=nd.arange(4)
print(w0.shape)
#对w进行reshape，
w=nd.arange(4).reshape((1,1,2,2))
print(w.shape)
b=nd.array([1])
print('b',b.shape)

(4,)
(1, 1, 2, 2)
b (1,)


In [2]:
print(w.shape[2:])
print(w.shape[0:])
print(w.shape[1:])

(2, 2)
(1, 1, 2, 2)
(1, 2, 2)


In [3]:
data=nd.arange(9).reshape((1,1,3,3))
out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[1])
print("data",data)
print("weight",w)
print("b",b)
print("out",out)

data 
[[[[ 0.  1.  2.]
   [ 3.  4.  5.]
   [ 6.  7.  8.]]]]
<NDArray 1x1x3x3 @cpu(0)>
weight 
[[[[ 0.  1.]
   [ 2.  3.]]]]
<NDArray 1x1x2x2 @cpu(0)>
b 
[ 1.]
<NDArray 1 @cpu(0)>
out 
[[[[ 20.  26.]
   [ 38.  44.]]]]
<NDArray 1x1x2x2 @cpu(0)>


In [4]:
out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[1],
                  stride=(2,2),pad=(1,1))

In [5]:
print('out',out)

out 
[[[[  1.   9.]
   [ 22.  44.]]]]
<NDArray 1x1x2x2 @cpu(0)>


## 输入有两个通道的情况

In [6]:
#weight也成了2个通道，分别对输入的不同通道 batch*channel*height*width
w=nd.arange(8).reshape((1,2,2,2))
#18个数据，分为两个通道，每个通道是3*3的大小。
data=nd.arange(18).reshape((1,2,3,3))
print('input:',data)

input: 
[[[[  0.   1.   2.]
   [  3.   4.   5.]
   [  6.   7.   8.]]

  [[  9.  10.  11.]
   [ 12.  13.  14.]
   [ 15.  16.  17.]]]]
<NDArray 1x2x3x3 @cpu(0)>


In [7]:
print(w)
print(b)


[[[[ 0.  1.]
   [ 2.  3.]]

  [[ 4.  5.]
   [ 6.  7.]]]]
<NDArray 1x2x2x2 @cpu(0)>

[ 1.]
<NDArray 1 @cpu(0)>


In [8]:
# 在通过权值与输入中对应通道的乘积计算后，再将各个通道结果相加。
out=nd.Convolution(data,w,b,kernel=w.shape[2:],num_filter=w.shape[0])
print(out)
print(out.shape)


[[[[ 269.  297.]
   [ 353.  381.]]]]
<NDArray 1x1x2x2 @cpu(0)>
(1, 1, 2, 2)


# 池化层

In [9]:
#输入数据为两个通道，3*3的height*width
data=nd.arange(18).reshape((1,2,3,3))
#对于每个层中的数据，分别做kernel的最大池化和平均池化。
max_pool=nd.Pooling(data=data,pool_type="max",kernel=(2,2))
avg_pool=nd.Pooling(data=data,pool_type="avg",kernel=(2,2))
print('max_pool',max_pool)
print('avg_pool',avg_pool)


max_pool 
[[[[  4.   5.]
   [  7.   8.]]

  [[ 13.  14.]
   [ 16.  17.]]]]
<NDArray 1x2x2x2 @cpu(0)>
avg_pool 
[[[[  2.   3.]
   [  5.   6.]]

  [[ 11.  12.]
   [ 14.  15.]]]]
<NDArray 1x2x2x2 @cpu(0)>


In [10]:
# 使用fashion MNIST
import sys
sys.path.append('..')
from utils import load_data_fashion_mnist

In [11]:
#一个批次中放入256个样本
batch_size=256
train_data,test_data=load_data_fashion_mnist(batch_size)

In [12]:
import mxnet as mx
try:
    ctx=mx.gpu()
    _=nd.zeros((1,),ctx=ctx)
except:
    ctx=mx.cpu()
ctx

gpu(0)

In [13]:
#进行一些初始化
weight_scale=0.01
#先两个卷积层，再两个全链接..
# 1是输入一个样本的特征，20是下一层的输入/// output channel 20 
W1=nd.random_normal(shape=(20,1,5,5),scale=weight_scale,ctx=ctx)
b1=nd.zeros(W1.shape[0],ctx=ctx)

W2=nd.random_normal(shape=(50,20,3,3),scale=weight_scale,ctx=ctx)
b2=nd.zeros(W2.shape[0],ctx=ctx)
#全连接
W3=nd.random_normal(shape=(1250,128),scale=weight_scale,ctx=ctx)
b3=nd.zeros(W3.shape[1],ctx=ctx)

W4=nd.random_normal(shape=(128,10),scale=weight_scale,ctx=ctx)
b4=nd.zeros(W4.shape[1],ctx=ctx)

params=[W1,b1,W2,b2,W3,b3,W4,b4]
for param in params:
    param.attach_grad()#前向传播

In [14]:
print('W1.shape0',W1.shape[0])

W1.shape0 20


In [30]:
#(256,1,28,28)中,256是一个Batch。
for i,_ in test_data:
    x=i.as_in_context(W1.context)
#X=.as_in_context(W1.context)
print(x.shape)
print(type(x))
print(W1.shape[2:])

(256, 1, 28, 28)
<class 'mxnet.ndarray.ndarray.NDArray'>
(5, 5)


In [31]:
#模型的定义，尝试修改卷积的filter和kernel
def net(X,verbose=False):
    X=X.as_in_context(W1.context)
    #第一层卷积
    h1_conv=nd.Convolution(data=X,weight=W1,bias=b1,kernel=W1.shape[2:],num_filter=W1.shape[0])
    h1_activation=nd.relu(h1_conv)
    h1=nd.Pooling(data=h1_activation,pool_type="avg",kernel=(2,2),stride=(2,2))
    
    #第二层卷积
    h2_conv=nd.Convolution(data=h1,weight=W2,bias=b2,kernel=W2.shape[2:],num_filter=W2.shape[0])
    h2_activation=nd.relu(h2_conv)
    h2=nd.Pooling(data=h2_activation,pool_type="avg",kernel=(2,2),stride=(2,2))
    h2=nd.flatten(h2)
    
    #第一层全连接
    h3_linear=nd.dot(h2,W3)+b3
    h3=nd.relu(h3_linear)
    
    #第二层全连接
    h4_linear=nd.dot(h3,W4)+b4
    
    if verbose:
        print("1st conv block: ",h1.shape)
        print("2st conv block: ",h2.shape)
        print("1st dense: ",h3.shape)
        print("2st dense: ",h4_linear.shape)
        print("output: ",h4_linear)
    return h4_linear

In [29]:
#为啥不是(256,20,28,28)啊
for data,_ in train_data:
    net(data,verbose=True)
    break

1st conv block:  (256, 20, 12, 12)
2st conv block:  (256, 1250)
1st dense:  (256, 128)
2st dense:  (256, 10)
output:  
[[ -6.22512744e-05   1.63621808e-05   6.10249772e-05 ...,   2.71430999e-05
    4.58553041e-05   7.30480024e-05]
 [ -4.98339796e-05   1.11888776e-05   4.82827681e-06 ...,   6.12251388e-05
    3.23877903e-05   4.35794100e-05]
 [ -7.63605640e-05   1.13514625e-05   3.74885021e-05 ...,   5.80633896e-05
    5.85587768e-05   6.18179547e-05]
 ..., 
 [ -2.21974497e-05   7.68643349e-06   1.64605135e-05 ...,   1.11172394e-05
    3.07310547e-05   3.03462530e-05]
 [ -8.21179201e-05  -2.61892856e-05  -5.09504798e-06 ...,   9.92904897e-05
    7.02863254e-05   6.86238855e-05]
 [ -1.14349124e-04  -1.04961482e-05   3.64272964e-05 ...,   9.49199894e-05
    7.25093269e-05   8.06859316e-05]]
<NDArray 256x10 @gpu(0)>


# 训练

In [19]:
from mxnet import autograd as autograd
from utils import SGD,accuracy,evaluate_accuracy
from mxnet import gluon
softmax_cross_entropy=gluon.loss.SoftmaxCrossEntropyLoss()
learning_rate=0.2

for epoch in range(5):
    train_loss=0.0
    train_acc=0.0
    for data,label in train_data:
        label=label.as_in_context(ctx)
        with autograd.record():
            output=net(data)
            loss=softmax_cross_entropy(output,label)
        loss.backward()
        SGD(params,learning_rate/batch_size)
        train_loss+=nd.mean(loss).asscalar()
        train_acc+=accuracy(output,label)
    test_acc=evaluate_accuracy(test_data,net,ctx)
    print("Epoch %d. Loss %f, Train acc %f, Test acc %f"%(
    epoch,train_loss/len(train_data),train_acc/len(train_data),test_acc))

MXNetError: [20:31:50] src/operator/./convolution-inl.h:446: Check failed: dshp.ndim() == 3U (4 vs. 3) Input data should be 3D in batch-num_filter-x

Stack trace returned 10 entries:
[bt] (0) /home/wuchunsheng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x26eb4c) [0x7f52e99e0b4c]
[bt] (1) /home/wuchunsheng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2766460) [0x7f52ebed8460]
[bt] (2) /home/wuchunsheng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x26cf4a7) [0x7f52ebe414a7]
[bt] (3) /home/wuchunsheng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2527362) [0x7f52ebc99362]
[bt] (4) /home/wuchunsheng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2513819) [0x7f52ebc85819]
[bt] (5) /home/wuchunsheng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x246658b) [0x7f52ebbd858b]
[bt] (6) /home/wuchunsheng/anaconda3/lib/python3.6/site-packages/mxnet/libmxnet.so(MXImperativeInvokeEx+0x63) [0x7f52ebbd8ac3]
[bt] (7) /home/wuch